# OpenAI Agent Query Planning
In this demo, we explore adding a QueryPlanTool to an OpenAIAgent. This effectively enables the agent to do advanced query planning, all through a single tool!

The QueryPlanTool is designed to work well with the OpenAI Function API. The tool takes in a set of other tools as input. The tool function signature contains of a QueryPlan Pydantic object, which can in turn contain a DAG of QueryNode objects defining a compute graph. The agent is responsible for defining this graph through the function signature when calling the tool. The tool itself executes the DAG over any corresponding tools.

In this setting we use a familiar example: Uber 10Q filings in March, June, and September of 2022.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Global Models


In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [ ]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI

In [ ]:
llm = OpenAI(temperature=0, model="gpt-4o-mini")

# Download Data


In [ ]:
os.makedirs('data/10q/', exist_ok=True)
!curl "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf" -o "./data/10q/uber_10q_march_2022.pdf"
!curl "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf" -o "./data/10q/uber_10q_june_2022.pdf"
!curl "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_sept_2022.pdf" -o "./data/10q/uber_10q_sept_2022.pdf"

#Load data


In [ ]:
march_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_june_2022.pdf"]
).load_data()
sept_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_sept_2022.pdf"]
).load_data()

# Build indices
We build a vector index / query engine over each of the documents (March, June, September).

In [ ]:
march_index = VectorStoreIndex.from_documents(march_2022)
june_index = VectorStoreIndex.from_documents(june_2022)
sept_index = VectorStoreIndex.from_documents(sept_2022)

In [ ]:
march_index.storage_context.persist(persist_dir="./march_index")
june_index.storage_context.persist(persist_dir="./june_2022")
sept_index.storage_context.persist(persist_dir="./sept_2022")

In [ ]:
march_engine = march_index.as_query_engine(similarity_top_k=3, llm=llm)
june_engine = june_index.as_query_engine(similarity_top_k=3, llm=llm)
sept_engine = sept_index.as_query_engine(similarity_top_k=3, llm=llm)

# OpenAI Function Agent with a Query Plan Tool
Use OpenAIAgent, built on top of the OpenAI tool use interface.

Feed it our QueryPlanTool, which is a Tool that takes in other tools. And the agent to generate a query plan DAG over these tools.

In [ ]:
from llama_index.core.tools import QueryEngineTool


query_tool_sept = QueryEngineTool.from_defaults(
    query_engine=sept_engine,
    name="sept_2022",
    description=(
        f"Provides information about Uber quarterly financials ending"
        f" September 2022"
    ),
)
query_tool_june = QueryEngineTool.from_defaults(
    query_engine=june_engine,
    name="june_2022",
    description=(
        f"Provides information about Uber quarterly financials ending"
        f" June 2022"
    ),
)
query_tool_march = QueryEngineTool.from_defaults(
    query_engine=march_engine,
    name="march_2022",
    description=(
        f"Provides information about Uber quarterly financials ending"
        f" March 2022"
    ),
)

In [ ]:
# define query plan tool
from llama_index.core.tools import QueryPlanTool
from llama_index.core import get_response_synthesizer

response_synthesizer = get_response_synthesizer()
query_plan_tool = QueryPlanTool.from_defaults(
    query_engine_tools=[query_tool_sept, query_tool_june, query_tool_march],
    response_synthesizer=response_synthesizer,
)

In [ ]:
query_plan_tool.metadata.to_openai_tool()  # to_openai_function() deprecated

#query_plan_tool.metadata.to_openai_function()

In [ ]:
query_plan_tool.metadata

In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI


agent = OpenAIAgent.from_tools(
    [query_plan_tool],
    max_function_calls=10,
    llm=OpenAI(temperature=0, model="gpt-4o-mini"),
    verbose=True,
)

In [ ]:
response = agent.query("What were the risk factors in march 2022?")

In [ ]:
response

In [ ]:
response = agent.query("What were the risk factors in sept 2022?")

In [ ]:
response

In [ ]:
from llama_index.core.tools.query_plan import QueryPlan, QueryNode

query_plan = QueryPlan(
    nodes=[
        QueryNode(
            id=1,
            query_str="risk factors",
            tool_name="sept_2022",
            dependencies=[],
        )
    ]
)

In [ ]:
QueryPlan.model_json_schema()

In [ ]:
response = agent.query(
    "Analyze Uber revenue growth in March, June, and September 2022"
)

In [ ]:
response

In [ ]:
print(str(response))

In [ ]:
response = agent.query(
    "Analyze changes in risk factors in march, june, and september for Uber"
)

In [ ]:
print(str(response))

In [ ]:
response = agent.query("Analyze both Uber revenue growth and risk factors over march, june, and september")

In [ ]:
print(str(response))

In [ ]:
response = agent.query(
    "First look at Uber's revenue growth and risk factors in March, "
    + "then revenue growth and risk factors in September, and then compare and"
    " contrast the two documents?"
)